In [1]:
#ライブラリーのインポート
from bs4 import BeautifulSoup
import re
import requests
from time import sleep
from tqdm import tqdm
import pandas as pd


#変数urlにSUUMOホームページのURLを格納する
url = 'https://suumo.jp/chintai/tokyo/sc_ota/mansion/?page={}'
#'Requests'を使って1で設定したURLにアクセスする
res = requests.get(url.format(1))
#取得したHTMLを'BeautifulSoup'で解析する
soup_total_page = BeautifulSoup(res.text, 'html.parser')
#最後のPage数を取得する
last_page = int(soup_total_page.find('ol', class_='pagination-parts').find_all('li')[-1].text)

#変数d_listにからのリストを作成する
d_list = []

#1〜3をループする
for i in tqdm(range(1, last_page+1)):
    
    print('d_listの大きさ:', len(d_list))
    
    #変数target_urlに、アクセスさくのURLを格納する
    target_url = url.format(i)
    #print()してtarget_urlを確認する
    print(target_url)
    
    #2. 'Requests'を使って1で設定したURLにアクセスする
    r = requests.get(target_url)
    
    #requestsの間隔を1秒あける
    sleep(1)

    #3. 取得したHTMLを'BeautifulSoup'で解析する
    soup = BeautifulSoup(r.text, 'html.parser')

    #4. 全ての物件情報(20件)を取得する。
    contents = soup.find_all('div', class_='cassetteitem')
    
    #5. 各物件情報から『物件の詳細』と『各部屋の情報』
    #各物件情報をforループで取得する。
    for content in contents:
        #6. 物件情報と部屋情報を取得しておく
        #物件情報を変数detailに格納する
        #各部屋の情報を変数tableに格納する
        detail = content.find('div', class_='cassetteitem-detail')
        table = content.find('table', class_='cassetteitem_other')
    
        #物件情報から必要な情報を取得する。
        #変数titleに物件名を格納する
        #変数addressに住所を格納する
        #変数accessにアクセス情報を格納する
        #変数age_building_floorsに築年数と建物の階数を格納する
        title = detail.find('div', class_='cassetteitem_content-title').text
        address = detail.find('li', class_='cassetteitem_detail-col1').text
        access = detail.find('li', class_='cassetteitem_detail-col2')
        age_building_floors = detail.find('li', class_='cassetteitem_detail-col3')
        
        #accessについて詳細に情報を取得していく
        #access要素内のすべてのdivタグを取得
        access_divs = access.find_all('div')
        
        
        # 路線名と駅名を取得する正規表現パターン
        station_pattern = r'(.+?/.*?駅)'
        # 数字のみを抽出する正規表現パターン
        pattern = r'\d+'

        # 各最寄駅の情報を格納するためのリスト
        stations = []
        times = []

        # access要素内のすべてのdivタグを取得し、ループ処理
        for access_div in access.find_all('div')[:3]:  # 最初の3つのdivタグのみを対象
            access_text = access_div.text

            # 駅名を抽出
            station_match = re.search(station_pattern, access_text)
            if station_match:
                station = station_match.group(1) + '駅'
            else:
                station = None

            # 最寄駅からの時間を抽出
            time_match = re.search(pattern, access_text)
            if time_match:
                time = int(time_match.group())
            else:
                time = 0

            # 結果をリストに追加
            stations.append(station)
            times.append(time)

        # 最寄駅名と時間の情報を変数に展開
        access_1_station, access_2_station, access_3_station = stations
        access_1_time, access_2_time, access_3_time = times
         
        #age_building_floors要素内のすべてのdivタグを取得
        age_building_floors_divs = age_building_floors.find_all('div')

        # 築年数age_textと建物の階数building_floors_textのテキストを取得
        age_text = age_building_floors_divs[0].text 
        building_floors_text = age_building_floors_divs[1].text 
        
        #数字のみを抽出する正規表現パターン
        pattern = r'\d+'
        #築年数から数字を抽出
        age = re.search(pattern, age_text)
        if age:
            age = age.group()
        #建物の階数から数字を抽出
        building_floors = re.search(pattern, building_floors_text)
        if building_floors:
            building_floors = building_floors.group()
    
        #部屋情報のブロックから各部屋情報を取得する
        #変数tableから全てのtrタグを取得して、変数tr_tagsに格納
        #tr_tagsの中から最初の1つだけtr_tagに格納
        tr_tags = table.find_all('tr', class_='js-cassette_link')
        tr_tag = tr_tags[0]
    
        #各部屋情報をforループで取得する
        for tr_tag in tr_tags:
        
            #部屋情報の行から、欲しい情報を取得する
            #変数floor, price, first_fee, capacityに4つの情報を格納する
            floor_number, price, first_fee, capacity = tr_tag.find_all("td")[2:6]
            #さらに細かい情報を取得する
            #priceは賃料(fee)と管理費(management_fee)
            #first_feeは敷金(deposit)と礼金(qratuity)
            #capacityは間取り(madori)と専有面積(menseki)
            rent_price, manegement_fee_price = price.find_all('li')
            deposit_price, qratuity_price = first_fee.find_all('li')
            madori_capacity, menseki_capacity = capacity.find_all('li')
            
            #テキストデータを取得する
            floor_text = floor_number.text
            rent_text = rent_price.text
            manegement_fee_text = manegement_fee_price.text
            deposit_text = deposit_price.text
            qratuity_text = qratuity_price.text
            madori = madori_capacity.text
            menseki_text = menseki_capacity.text
            
            #数字のみを抽出する正規表現パターン
            pattern = r'\d+'
            
            # 小数を含む数値にマッチする正規表現パターン
            pattern2 = r'\d+(\.\d+)?'
            
            #階数floor_textから数字を抽出
            #int型にして、階数floorに格納する。
            floor = re.search(pattern, floor_text)
            if floor:
                floor = int(floor.group())
            
            #管理費manegement_fee_textから数字を抽出
            #int型にして、管理費manegement_fee_textに格納する。
            manegement_fee = re.search(pattern, manegement_fee_text)
            if manegement_fee:
                manegement_fee = int(manegement_fee.group())
            else:
                manegement_fee = 0  # 管理費が取得できない場合は0円とする

            
            
            #面積mensekiから数字を抽出
            #float型にして、面積mensekiに格納する。
            menseki = re.search(pattern2, menseki_text)
            if menseki:
                menseki = float(menseki.group())
        
            #数字（整数または小数）を抽出する正規表現パターン
            pattern3 = r'(\d+(\.\d+)?)万'
            
            # 金額を円単位に変換する関数
            def convert_to_yen(text):
                # '万'単位の金額を探す
                match = re.search(pattern3, text)
                if match:
                    # 数字部分を取得し、float型に変換してから10000を乗算
                    number = float(match.group(1))
                    return int(number * 10000)
                else:
                    #'万'がない場合は通常の数字を探す
                    match = re.search(r'\d+', text)
                    if match:
                        return int(match.group())
                    else:
                        # 数字が見つからない場合は0や適切なデフォルト値を返す
                        return 0                
            # 各テキストから金額を円単位で抽出
            rent = convert_to_yen(rent_text)
            deposit = convert_to_yen(deposit_text)
            qratuity = convert_to_yen(qratuity_text)

        
            #取得した全ての情報を辞書に格納する
            #変数dにこれまで取得した11項目を辞書に格納する
            d = {
            "物件名": title, 
            "住所": address,
            "アクセス": access,
            "最寄駅1": access_1_station,
            "最寄駅1からの時間(分)": access_1_time,
            "最寄駅2": access_2_station,
            "最寄駅2からの時間(分)": access_2_time,
            "最寄駅3": access_3_station,
            "最寄駅3からの時間(分)": access_3_time,
            "築年数(年)": age,
            "建物の階数": building_floors,
            "階数": floor,
            "家賃(円)": rent,
            "管理費(円)": manegement_fee,
            "敷金(円)": deposit,
            "礼金(円)": qratuity,
            "間取り": madori,
            "面積(m2)": menseki,
            }
        
            #取得した辞書をd_listに格納する
            d_list.append(d)
            
#変数d_listを使って、データフレームを作成する
df = pd.DataFrame(d_list)

# この関数はDataFrameの各セルをクリーニングするために使用されます
def clean_text(cell):
    # セルがNoneの場合、変更せずにそのまま返します
    if cell is None:
        return cell

    # セルが文字列型の場合、改行、キャリッジリターン、タブをスペースに置換して返します
    if isinstance(cell, str):
        return re.sub('[\n\r\t]', ' ', cell)

    # セルがBeautifulSoupのTagオブジェクトの場合（HTMLタグ）、
    # そのテキスト内容を取得し、同様に置換して返します
    elif hasattr(cell, 'text'):
        return re.sub('[\n\r\t]', ' ', cell.text)

    # 上記の条件に当てはまらない場合、セルをそのまま返します
    else:
        return cell

# DataFrameの各セルに対してclean_text関数を適用します
df2 = df.applymap(clean_text)

#住所、家賃、階が一致した物件を重複物件とみなし、重複した行を特定する条件を指定
duplicate_condition = df2.duplicated(subset=['住所', '家賃(円)', '階数', '間取り', '面積(m2)'], keep=False)

# 重複した行を抽出
duplicates = df2[duplicate_condition]

# 重複した行を削除
df3 = df2.drop_duplicates(subset=['住所', '家賃(円)', '階数',  '間取り', '面積(m2)'], keep='first')

#df3をcsvに出力
df3.to_csv('SUUMO_Otaku2.csv', index=False, encoding='utf-8-sig')

  0%|          | 0/316 [00:00<?, ?it/s]

d_listの大きさ: 0
https://suumo.jp/chintai/tokyo/sc_ota/mansion/?page=1


  0%|          | 1/316 [00:02<12:06,  2.30s/it]

d_listの大きさ: 80
https://suumo.jp/chintai/tokyo/sc_ota/mansion/?page=2


  1%|          | 2/316 [00:04<11:57,  2.29s/it]

...

 99%|█████████▉| 314/316 [11:25<00:04,  2.23s/it]

d_listの大きさ: 12633
https://suumo.jp/chintai/tokyo/sc_ota/mansion/?page=315


100%|█████████▉| 315/316 [11:28<00:02,  2.18s/it]

d_listの大きさ: 12663
https://suumo.jp/chintai/tokyo/sc_ota/mansion/?page=316


100%|██████████| 316/316 [11:30<00:00,  2.18s/it]


In [3]:
#Google spredsheets関連のライブラリーをインポート
import gspread
from oauth2client.service_account import ServiceAccountCredentials
#環境変数関連
from dotenv import load_dotenv
load_dotenv()
import os


#スコープとjsonファイルを使って認証情報を取得
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = os.getenv('SERVICE_ACCOUNT_FILE')
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, SCOPES)
#認証情報をauthorize関数に渡してスプレッドシートの操作権を取得
gs = gspread.authorize(credentials)

#シート情報を取得して変数に代入
SPREADSHEET_KEY = os.getenv('SPREADSHEET_KEY')
workbook = gs.open_by_key(SPREADSHEET_KEY)

# スプレッドシートを開く（新規作成または既存のものを開く）
spreadsheet = gs.open('Step3-1②-1')

# ワークシートを取得（新規作成または既存のものを開く）
worksheet = spreadsheet.get_worksheet(0) # 0は最初のワークシートを指します

# データフレームをGoogleスプレッドシートに転送
worksheet.update([df3.columns.values.tolist()] + df3.values.tolist())

/var/folders/3x/2wmnmhys6_n8bjxz4xywkhb40000gn/T/ipykernel_69406/2535239105.py:28: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df3.columns.values.tolist()] + df3.values.tolist())


{'spreadsheetId': '1bDCqElveejgA3h2keb2wAGEqQ8eSsJexOAxAsh_hGeE',
 'updatedRange': "'シート1'!A1:R8165",
 'updatedRows': 8165,
 'updatedColumns': 18,
 'updatedCells': 145042}

In [4]:
import sqlite3

# SQLiteデータベースファイルを作成（または既存のものに接続）
conn = sqlite3.connect('SUUMO_Otaku_database.db')

# このデータフレームを'20231212_SUUMO_Otaku'という名前のテーブルに転送
# 存在しない場合は新規作成、存在する場合は置き換える
df3.to_sql('20231212_SUUMO_Otaku', conn, if_exists='replace', index=False)

# データベースの接続を閉じる
conn.close()